In [1]:
import os,time,cv2, sys, math
import tensorflow as tf
import argparse
import numpy as np
import os
from utils import utils, helpers
from builders import model_builder

# sepcify the gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '/device:GPU:0'
# hyper-parameters setting
parser = argparse.ArgumentParser()
parser.add_argument('--image', type=str, default="./images/0016E5_08147_L.png", help='The image you want to predict on.')
parser.add_argument('--checkpoint_path', type=str, default="./checkpoints/latest_model_BiSeNet_CamVid.ckpt", help='The path to the latest checkpoint weights for your model.')
parser.add_argument('--crop_height', type=int, default=640, help='Height of cropped input image to network')
parser.add_argument('--crop_width', type=int, default=800, help='Width of cropped input image to network')
parser.add_argument('--model', type=str, default="BiSeNet", help='The model you are using')
parser.add_argument('--dataset', type=str, default="CamVid", help='The dataset you are using')
args = parser.parse_args(args=[])

class_names_list, label_values = helpers.get_label_info(os.path.join(args.dataset, "class_dict.csv"))

num_classes = len(label_values)
infer_size = (800, 640)
print("\n***** Begin prediction *****")
print("Dataset -->", args.dataset)
print("Model -->", args.model)
print("Crop Height -->", args.crop_height)
print("Crop Width -->", args.crop_width)
print("Num Classes -->", num_classes)
print("Image -->", args.image)

# allow the use of gpu
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.Session(config=config)
# Use placeholders as formal parameters to reduce the op of the training process
net_input = tf.placeholder(tf.float32,shape=[None,None,None,3])
# load the network (BiseNet, depth-wise BiseNet, depth-wise AAFF, depth-wise AAFF2)
network, _ = model_builder.build_model(args.model, net_input=net_input,
                                        num_classes=num_classes,
                                        crop_width=args.crop_width,
                                        crop_height=args.crop_height,
                                        is_training=False)

sess.run(tf.global_variables_initializer())

print('Loading model checkpoint weights')
saver=tf.train.Saver(max_to_keep=500)
# load the model weights
saver.restore(sess, args.checkpoint_path)
# load the target image for prediction
loaded_image = utils.load_image(args.image)
resized_image =cv2.resize(loaded_image, (args.crop_width, args.crop_height))
input_image = np.expand_dims(np.float32(resized_image[:args.crop_height, :args.crop_width]),axis=0)/255.0
# perform prediction
output_image = sess.run(network, feed_dict={net_input:input_image})

# compute the inference speed
elapse = []
for i in range(50):
    start = time.time()
    output_image = sess.run(network, feed_dict={net_input:input_image})
    duration = time.time() - start
    print('time: {:.4f}, about {:.6f} fps'.format(duration, 1 / duration))
    elapse.append(duration)
print('Average time: {:.4f}, about {:.6f} fps'.format(np.mean(elapse), 1 / np.mean(elapse)))

# output a colored predicted image
output_image = np.array(output_image[0,:,:,:])
output_image = helpers.reverse_one_hot(output_image)
out_vis_image = helpers.colour_code_segmentation(output_image, label_values)
file_name = utils.filepath_to_name(args.image)
cv2.imwrite("%s_pred.png"%(file_name),cv2.cvtColor(np.uint8(out_vis_image), cv2.COLOR_RGB2BGR))

print("")
print("Finished!")
print("Wrote image " + "%s_pred.png"%(file_name))


C:\Users\Administrator\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Administrator\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Administrator\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Administrator


***** Begin prediction *****
Dataset --> CamVid
Model --> BiSeNet
Crop Height --> 640
Crop Width --> 800
Num Classes --> 32
Image --> ./images/0016E5_08147_L.png
Preparing the model ...
Loading model checkpoint weights
INFO:tensorflow:Restoring parameters from ./checkpoints/latest_model_BiSeNet_CamVid.ckpt
time: 2.1301, about 0.469457 fps
time: 2.4781, about 0.403528 fps
time: 2.0621, about 0.484938 fps
time: 2.0411, about 0.489928 fps
time: 2.3091, about 0.433063 fps
Average time: 2.2041, about 0.453695 fps

Finished!
Wrote image 0016E5_08147_L_pred.png


In [2]:
import tensorflow as tf
print(tf.__version__)

1.4.0
